# LangChain Agent Executor
In this chapter, we will continue from the introduction to agents and dive deeper into agents. Learning how to build our custom agent execution loop for v0.3 of LangChain.

![Agent Executor](images/agentExecutor.png)

We have user input that goes into LLm then...

![Agent Executor](images/agentExecutor2.png)

# Tools
As with the previous chapter, we will define a few tools to use within our agent.

In [34]:
from langchain_core.tools import tool

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

# Define the multiply tool
@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' and 'y'."""
    return x * y

# Define the exponentiate tool
@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'."""
    return x ** y

@tool
def subtract(x: float, y: float) -> float:
    """Subtract 'x' from 'y'."""
    return y - x

In [35]:
add

StructuredTool(name='add', description="Add 'x' and 'y'.", args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x00000268F4403420>)

In [38]:
print(f"{add.name=}\n{add.description=}")

add.name='add'
add.description="Add 'x' and 'y'."


In [40]:
add.args_schema.model_json_schema()

{'description': "Add 'x' and 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'add',
 'type': 'object'}

In [41]:
exponentiate.args_schema.model_json_schema()

{'description': "Raise 'x' to the power of 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'exponentiate',
 'type': 'object'}

In [42]:
import json

llm_output_string = "{\"x\": 5, \"y\": 2}"  # this is the output from the LLM
llm_output_dict = json.loads(llm_output_string)  # load as dictionary
llm_output_dict


exponentiate.func(**llm_output_dict)

25

# Creating an Agent
We will use LangChain Epression Language (LCEL) to construct the agent. We will cover LCEL more in the next chapter, but for now - all we need to know is that our agent will be constructed using syntax and components like so:

In [43]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You're a helpful assistant. When answering a user's question "
        "you should first use one of the tools provided. After using a "
        "tool the tool output will be provided in the "
        "'scratchpad' below. If you have an answer in the "
        "scratchpad you should not use any more tools and "
        "instead answer directly to the user."
    )),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [45]:
from langchain_ollama.chat_models import ChatOllama

model_name = "llama3.1"
# smaller alternative: llama3.2:3b-instruct-fp16

# initialize one LLM with temperature 0.0, this makes the LLM more deterministic
llm = ChatOllama(model=model_name, temperature=0.0)

In [46]:
from langchain_core.runnables.base import RunnableSerializable

tools = [add, subtract, multiply, exponentiate]

# define the agent runnable
agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="any")
)

In [47]:
tool_call = agent.invoke({"input": "What is 10 + 10", "chat_history": []})
tool_call

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-05-15T08:26:48.2101241Z', 'done': True, 'done_reason': 'stop', 'total_duration': 15110344300, 'load_duration': 4349157800, 'prompt_eval_count': 411, 'prompt_eval_duration': 1101354900, 'eval_count': 46, 'eval_duration': 9655095000, 'model_name': 'llama3.1'}, id='run--b792435b-e437-45b7-8079-fa3beff8828c-0', tool_calls=[{'name': 'add', 'args': {'x': 10, 'y': 10}, 'id': '889f7562-d97a-4821-a0a1-849ac2ccbd1d', 'type': 'tool_call'}, {'name': 'add', 'args': {'x': 10, 'y': 10}, 'id': 'fd4502a2-d617-4de7-bcfb-f10757329af8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 411, 'output_tokens': 46, 'total_tokens': 457})

In [50]:
tool_call.tool_calls

[{'name': 'add',
  'args': {'x': 10, 'y': 10},
  'id': '889f7562-d97a-4821-a0a1-849ac2ccbd1d',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'x': 10, 'y': 10},
  'id': 'fd4502a2-d617-4de7-bcfb-f10757329af8',
  'type': 'tool_call'}]

In [49]:
# create tool name to function mapping
name2tool = {tool.name: tool.func for tool in tools}

In [51]:
tool_exec_content = name2tool[tool_call.tool_calls[0]["name"]](
    **tool_call.tool_calls[0]["args"]
)
tool_exec_content

20

In [52]:
from langchain_core.messages import ToolMessage

tool_exec = ToolMessage(
    content=f"The {tool_call.tool_calls[0]['name']} tool returned {tool_exec_content}",
    tool_call_id=tool_call.tool_calls[0]["id"]
)

out = agent.invoke({
    "input": "What is 10 + 10",
    "chat_history": [],
    "agent_scratchpad": [tool_call, tool_exec]
})
out

AIMessage(content='The answer to the question is 20.', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-05-15T08:28:37.8233646Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2299329400, 'load_duration': 25794600, 'prompt_eval_count': 181, 'prompt_eval_duration': 632601000, 'eval_count': 10, 'eval_duration': 1637171200, 'model_name': 'llama3.1'}, id='run--42285fd8-e4cb-45c6-9e60-0d3a29639cf8-0', usage_metadata={'input_tokens': 181, 'output_tokens': 10, 'total_tokens': 191})

In [53]:
agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="auto")
)

In [54]:
tool_call = agent.invoke({"input": "What is 10 + 10", "chat_history": []})
tool_call

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-05-15T08:29:48.9015028Z', 'done': True, 'done_reason': 'stop', 'total_duration': 10658337400, 'load_duration': 18766800, 'prompt_eval_count': 411, 'prompt_eval_duration': 956053200, 'eval_count': 46, 'eval_duration': 9681283100, 'model_name': 'llama3.1'}, id='run--6479c606-0172-4d7f-a79d-42163ad3ccaf-0', tool_calls=[{'name': 'add', 'args': {'x': 10, 'y': 10}, 'id': '5736edfe-743a-4ed7-8a0f-69710db121c2', 'type': 'tool_call'}, {'name': 'add', 'args': {'x': 10, 'y': 10}, 'id': '102b6ad7-943b-4cb0-b477-de98b2264974', 'type': 'tool_call'}], usage_metadata={'input_tokens': 411, 'output_tokens': 46, 'total_tokens': 457})

In [55]:
tool_output = name2tool[tool_call.tool_calls[0]["name"]](
    **tool_call.tool_calls[0]["args"]
)

tool_exec = ToolMessage(
    content=f"The {tool_call.tool_calls[0]['name']} tool returned {tool_output}",
    tool_call_id=tool_call.tool_calls[0]["id"]
)

out = agent.invoke({
    "input": "What is 10 + 10",
    "chat_history": [],
    "agent_scratchpad": [tool_call, tool_exec]
})
out

AIMessage(content='The answer to the question is 20.', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-05-15T08:30:03.8385916Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2376580000, 'load_duration': 17710200, 'prompt_eval_count': 181, 'prompt_eval_duration': 685940200, 'eval_count': 10, 'eval_duration': 1671405000, 'model_name': 'llama3.1'}, id='run--d237fd5a-beea-48d9-9c54-21dd4e688360-0', usage_metadata={'input_tokens': 181, 'output_tokens': 10, 'total_tokens': 191})

In [56]:
@tool
def final_answer(answer: str, tools_used: list[str]) -> str:
    """Use this tool to provide a final answer to the user.
    The answer should be in natural language as this will be provided
    to the user directly. The tools_used must include a list of tool
    names that were used within the `scratchpad`.
    """
    return {"answer": answer, "tools_used": tools_used}

In [57]:
tools = [final_answer, add, subtract, multiply, exponentiate]

# we need to update our name2tool mapping too
name2tool = {tool.name: tool.func for tool in tools}

agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="any")  # we're forcing tool use again
)

In [58]:
tool_call = agent.invoke({"input": "What is 10 + 10", "chat_history": []})
tool_call.tool_calls

[{'name': 'add',
  'args': {'x': 10, 'y': 10},
  'id': 'edfcf7cb-93a3-4eb3-82c4-2f4bff26c4c7',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'x': 10, 'y': 10},
  'id': '3188d9b9-14ce-4811-8e1d-2dede6519616',
  'type': 'tool_call'}]

In [61]:
tool_call = agent.invoke({"input": "What is 10 + 10", "chat_history": []})
tool_call.tool_calls[0]["args"]

{'x': 10, 'y': 10}

In [59]:
tool_out = name2tool[tool_call.tool_calls[0]["name"]](
    **tool_call.tool_calls[0]["args"]
)

tool_exec = ToolMessage(
    content=f"The {tool_call.tool_calls[0]['name']} tool returned {tool_out}",
    tool_call_id=tool_call.tool_calls[0]["id"]
)

out = agent.invoke({
    "input": "What is 10 + 10",
    "chat_history": [],
    "agent_scratchpad": [tool_call, tool_exec]
})
out

AIMessage(content='The answer to the question is 20.', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-05-15T08:31:37.7203508Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2488898800, 'load_duration': 17413600, 'prompt_eval_count': 181, 'prompt_eval_duration': 675778100, 'eval_count': 10, 'eval_duration': 1793923600, 'model_name': 'llama3.1'}, id='run--0398bd04-4e7b-44fa-abad-1f46f8a3abd9-0', usage_metadata={'input_tokens': 181, 'output_tokens': 10, 'total_tokens': 191})

In [60]:
out.tool_calls

[]